In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from TO_sim.Sol_Kuramoto import Sol_Kuramoto_mf2 as mf2
from TO_sim.Sol_Kuramoto import Sol_Kuramoto_mf2_sets as mf2_sets
from TO_sim.get_cluster import C_rsmso,cluster_os,C_rsmso_set_new
from TO_sim.To_Draw import Draw_avg_vel_r

from TO_sim.gen_Distribution import Normal
from TO_sim.gen_Distribution import Quantile_Normal as Q_Normal
import parmap
from scipy.signal import find_peaks
from TO_sim.To_Draw import Draw_avg_vel_r
from tqdm.notebook import tqdm


In [3]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [4]:
df = pd.read_excel('N = 1000 Q_Normal seed = 10 dt = 0.1 230821 ver3.xlsm',index_col=[0,1])
ms = df.reset_index().m.unique()

In [5]:
dK = 0.1
dm = 0.1
N = 1000
seed = 10
dt = 0.1
theta_init,omega,Kc = Q_Normal(N,0,1,seed=seed)
K_set = np.arange(0,5,dK).reshape(-1,1)

N_set = len(K_set)
omega = np.sort(omega) - np.mean(omega)
ms = np.arange(0,10+dm/2,dm)
m_set = []
core = 15
index_ = 0
for i in range(len(ms)//core +1):
    m_set.append(ms[index_:index_+core])
    index_ += core
core_set = [len(ms) for ms in m_set]
N_set = len(K_set)
theta_init_set = (np.ones((N_set,1))*theta_init)
pdtheta_set = 0*(np.ones((N_set,1))*omega)
omega_set = (np.ones((N_set,1))*omega)

In [6]:
core_set

[15, 15, 15, 15, 15, 15, 11]

In [7]:
cols ={'r':[0],'rstd':[0],'rMM':[0],'error':[0]}
for i in range(10):
    cols[f'c{i}'] = np.nan
for i in range(10):
    cols[f'c{i} phase vel'] = np.nan
for i in range(10):
    cols[f'c{i} omega'] = np.nan

cols['K'] = K_set.reshape(-1)[0]
cols['m'] = ms[0]
df = pd.DataFrame(cols)
df_Km = df.set_index(['K','m'])

In [9]:
df_Km = df.copy()

In [8]:
for core,ms in zip(core_set,m_set):
    for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                        omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=dt,
                                                        pm_pbar=True, pm_processes=core)):
        for i,K in enumerate(K_set.reshape(-1)):
            df_Km.loc[(K,m),"r"] = r[i]
            df_Km.loc[(K,m),"rstd"] = rstd[i]
            df_Km.loc[(K,m),"rMM"] = rMM[i]
            df_Km.loc[(K,m),'error'] = error_set[i]
            CSMP = CSMP_set[i]
            CSMO = CSMO_set[i]
            for i in range(len(CSMP_set[i])):
                    df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                    df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                    df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
core_set

[15, 15, 15, 15, 1]

In [10]:
K_set = np.arange(4,10+dK/2,dK).reshape(-1,1)
N_set = len(K_set)
omega = np.sort(omega) - np.mean(omega)
ms = np.arange(4,10+dm/2,dm)
m_set = []
core = 15
index_ = 0
for i in range(len(ms)//core +1):
    m_set.append(ms[index_:index_+core])
    index_ += core
core_set = [len(ms) for ms in m_set]
N_set = len(K_set)
theta_init_set = (np.ones((N_set,1))*theta_init)
pdtheta_set = 0*(np.ones((N_set,1))*omega)
omega_set = (np.ones((N_set,1))*omega)
for core,ms in zip(core_set,m_set):
    for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                        omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=5000,dt=dt,
                                                        pm_pbar=True, pm_processes=core)):
        for i,K in enumerate(K_set.reshape(-1)):
            df_Km.loc[(K,m),"r"] = r[i]
            df_Km.loc[(K,m),"rstd"] = rstd[i]
            df_Km.loc[(K,m),"rMM"] = rMM[i]
            df_Km.loc[(K,m),'error'] = error_set[i]
            CSMP = CSMP_set[i]
            CSMO = CSMO_set[i]
            for i in range(len(CSMP_set[i])):
                    df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                    df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                    df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
K_set

array([[ 4. ],
       [ 4.1],
       [ 4.2],
       [ 4.3],
       [ 4.4],
       [ 4.5],
       [ 4.6],
       [ 4.7],
       [ 4.8],
       [ 4.9],
       [ 5. ],
       [ 5.1],
       [ 5.2],
       [ 5.3],
       [ 5.4],
       [ 5.5],
       [ 5.6],
       [ 5.7],
       [ 5.8],
       [ 5.9],
       [ 6. ],
       [ 6.1],
       [ 6.2],
       [ 6.3],
       [ 6.4],
       [ 6.5],
       [ 6.6],
       [ 6.7],
       [ 6.8],
       [ 6.9],
       [ 7. ],
       [ 7.1],
       [ 7.2],
       [ 7.3],
       [ 7.4],
       [ 7.5],
       [ 7.6],
       [ 7.7],
       [ 7.8],
       [ 7.9],
       [ 8. ],
       [ 8.1],
       [ 8.2],
       [ 8.3],
       [ 8.4],
       [ 8.5],
       [ 8.6],
       [ 8.7],
       [ 8.8],
       [ 8.9],
       [ 9. ],
       [ 9.1],
       [ 9.2],
       [ 9.3],
       [ 9.4],
       [ 9.5],
       [ 9.6],
       [ 9.7],
       [ 9.8],
       [ 9.9],
       [10. ]])

In [11]:
f'N = {N} seed = {seed} dt = {dt} 230708 ver3.xlsm'

'N = 1000 seed = 10 dt = 0.1 230708 ver3.xlsm'

In [33]:
df_Km.sort_index()

r      rstd       rMM  error     c0  c1  c2  c3  c4  c5  \
K    m                                                                      
0.0  0.0   0.026198  0.014298  0.089169    NaN    NaN NaN NaN NaN NaN NaN   
     0.1   0.026197  0.014298  0.089169    NaN    NaN NaN NaN NaN NaN NaN   
     0.2   0.026197  0.014298  0.089169    NaN    NaN NaN NaN NaN NaN NaN   
     0.3   0.026195  0.014298  0.089169    NaN    NaN NaN NaN NaN NaN NaN   
     0.4   0.026192  0.014299  0.089169    NaN    NaN NaN NaN NaN NaN NaN   
...             ...       ...       ...    ...    ...  ..  ..  ..  ..  ..   
10.0 9.6   0.892879  0.005973  0.035209    NaN  909.0 NaN NaN NaN NaN NaN   
     9.7   0.891870  0.005588  0.030404    NaN  908.0 NaN NaN NaN NaN NaN   
     9.8   0.890725  0.006224  0.037913    NaN  907.0 NaN NaN NaN NaN NaN   
     9.9   0.889750  0.006621  0.039076    NaN  906.0 NaN NaN NaN NaN NaN   
     10.0  0.889907  0.005489  0.037329    NaN  906.0 NaN NaN NaN NaN NaN   

           ...  c0 omega  c1 omega  c2 omega  c3 omega  c4 omega  c5 omega  \
K    m     ...                                                               
0.0  0.0   ...       NaN       NaN       NaN       NaN       NaN       NaN   
     0.1   ...       NaN       NaN       NaN       NaN       NaN       NaN   
     0.2   ...       NaN       NaN       NaN       NaN       NaN       NaN   
     0.3   ...       NaN       NaN       NaN       NaN       NaN       NaN   
     0.4   ...       NaN       NaN       NaN       NaN       NaN       NaN   
...        ...       ...       ...       ...       ...       ...       ...   
10.0 9.6   ...  0.009942       NaN       NaN       NaN       NaN       NaN   
     9.7   ...  0.009942       NaN       NaN       NaN       NaN       NaN   
     9.8   ...  0.009942       NaN       NaN       NaN       NaN       NaN   
     9.9   ...  0.009942       NaN       NaN       NaN       NaN       NaN   
     10.0  ...  0.009942       NaN       NaN       NaN       NaN       NaN   

           c6 omega  c7 omega  c8 omega  c9 omega  
K    m                                             
0.0  0.0        NaN       NaN       NaN       NaN  
     0.1        NaN       NaN       NaN       NaN  
     0.2        NaN       NaN       NaN       NaN  
     0.3        NaN       NaN       NaN       NaN  
     0.4        NaN       NaN       NaN       NaN  
...             ...       ...       ...       ...  
10.0 9.6        NaN       NaN       NaN       NaN  
     9.7        NaN       NaN       NaN       NaN  
     9.8        NaN       NaN       NaN       NaN  
     9.9        NaN       NaN       NaN       NaN  
     10.0       NaN       NaN       NaN       NaN  

[13921 rows x 34 columns]

In [12]:
df_Km.sort_index().to_excel(f'N = {N} Q_Normal seed = {seed} dt = {dt} 230821 ver3.xlsm')

In [8]:
f'N = {N} Q_Normal seed = {seed} dt = {dt} 230821 ver3.xlsm'

'N = 500 Q_Normal seed = 10 dt = 0.1 230821 ver3.xlsm'

In [10]:
f'N = {N} seed = {seed} dt = {dt} 230720 ver3.xlsm'

'N = 500 seed = 10 dt = 0.1 230720 ver3.xlsm'

In [ ]:
for seed in [10]:
    dK = 0.1
    dm = 0.1
    N = 500

    theta_init,omega,Kc = Normal(N,0,1,seed=seed)
    K_set = np.arange(0,5,dK).reshape(-1,1)

    N_set = len(K_set)
    omega = np.sort(omega) - np.mean(omega)
    ms = np.arange(0,10+dm/2,dm)
    m_set = []
    core = 8
    index_ = 0
    for i in range(len(ms)//core +1):
        m_set.append(ms[index_:index_+core])
        index_ += core
    core_set = [len(ms) for ms in m_set]
    N_set = len(K_set)
    theta_init_set = (np.ones((N_set,1))*theta_init)
    pdtheta_set = 0*(np.ones((N_set,1))*omega)
    omega_set = (np.ones((N_set,1))*omega)
    core_set
    cols ={'r':[0],'rstd':[0],'rMM':[0],'error':[0]}
    for i in range(10):
        cols[f'c{i}'] = np.nan
    for i in range(10):
        cols[f'c{i} phase vel'] = np.nan
    for i in range(10):
        cols[f'c{i} omega'] = np.nan

    cols['K'] = K_set.reshape(-1)[0]
    cols['m'] = ms[0]
    df = pd.DataFrame(cols)
    df_Km = df.set_index(['K','m'])
    for core,ms in tqdm(list(zip(core_set,m_set))):pass
    for core,ms in zip(core_set,m_set):
        for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                            omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=0.1,
                                                            pm_pbar=True, pm_processes=core)):
            for i,K in enumerate(K_set.reshape(-1)):
                df_Km.loc[(K,m),"r"] = r[i]
                df_Km.loc[(K,m),"rstd"] = rstd[i]
                df_Km.loc[(K,m),"rMM"] = rMM[i]
                df_Km.loc[(K,m),'error'] = error_set[i]
                CSMP = CSMP_set[i]
                CSMO = CSMO_set[i]
                for i in range(len(CSMP_set[i])):
                        df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                        df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                        df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]
    K_set = np.arange(5,10+dK/2,dK).reshape(-1,1)
    N_set = len(K_set)
    omega = np.sort(omega) - np.mean(omega)
    ms = np.arange(0,10+dm/2,dm)
    m_set = []
    core = 8
    index_ = 0
    for i in range(len(ms)//core +1):
        m_set.append(ms[index_:index_+core])
        index_ += core
    core_set = [len(ms) for ms in m_set]
    N_set = len(K_set)
    theta_init_set = (np.ones((N_set,1))*theta_init)
    pdtheta_set = 0*(np.ones((N_set,1))*omega)
    omega_set = (np.ones((N_set,1))*omega)
    for core,ms in zip(core_set,m_set):
        for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                            omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=0.1,
                                                            pm_pbar=True, pm_processes=core)):
            for i,K in enumerate(K_set.reshape(-1)):
                df_Km.loc[(K,m),"r"] = r[i]
                df_Km.loc[(K,m),"rstd"] = rstd[i]
                df_Km.loc[(K,m),"rMM"] = rMM[i]
                df_Km.loc[(K,m),'error'] = error_set[i]
                CSMP = CSMP_set[i]
                CSMO = CSMO_set[i]
                for i in range(len(CSMP_set[i])):
                        df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                        df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                        df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]
    df_Km.sort_index().to_excel(f'N = {N} seed = {seed} test 230708 ver3.xlsm')
